In [1]:
from fishsense_api_workflow_worker.database import Database
from fishsense_api_workflow_worker.config import PG_CONNECTION_STRING

from sqlmodel.ext.asyncio.session import AsyncSession
from tqdm.notebook import tqdm

In [2]:
database = Database(PG_CONNECTION_STRING)

In [6]:
async with AsyncSession(database.engine) as session:
    for label in tqdm(await database.select_laser_labels()):
        if label.completed != True:
            continue

        original_width = label.json["annotations"][0]["result"][0]["original_width"]
        original_height = label.json["annotations"][0]["result"][0]["original_height"]

        x = label.json["annotations"][0]["result"][0]["value"]["x"] / 100
        y = label.json["annotations"][0]["result"][0]["value"]["y"] / 100

        x *= original_width
        y *= original_height

        label.x = x
        label.y = y

        label.label_studio_json = label.json

        await database.insert_or_update_laser_label(label, session=session)

    await session.commit()

  0%|          | 0/68217 [00:00<?, ?it/s]

In [7]:
async with AsyncSession(database.engine) as session:
    for label in tqdm(await database.select_head_tail_labels()):

        label.label_studio_json = label.json

        await database.insert_or_update_head_tail_label(label, session=session)

    await session.commit()

  0%|          | 0/41447 [00:00<?, ?it/s]